## Import Library

In [3]:
# Data Handling
import pandas as pd
import datetime as dt
import random
import itertools
# from selenium import webdriver
# from selenium.webdriver.common.by import By
import gc

# Data Visualizing
from tqdm import tqdm

#model
from sklearn.model_selection import train_test_split

# etc.
import warnings
warnings.filterwarnings('ignore')
import os
import time

DATA_PATH = 'C:/Users/USER/Desktop/Bbeojung.kr/Data'
CHROME_DRIVER = os.path.join(DATA_PATH, 'chromedriver_win32/chromedriver.exe')

## Download DataSet

In [27]:
# # Download df
# driver = webdriver.Chrome(CHROME_DRIVER)
# url = 'https://bigdata.gwangju.go.kr/usr/main/goMain.rd'
# driver.get(url)

# transp_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[3]/div/ul/li[18]/a')
# transp_btn.click()

# time.sleep(1)
# page_btn = driver.find_element(By.XPATH, '//*[@id="6"]')
# page_btn.click()

# for page in range(7,24):
#     for i in range(1,11):
#         time.sleep(1)
#         view_btn = driver.find_element(By.XPATH, f'//*[@id="dataSetListBody"]/tr[{i}]/td[8]/a')
#         view_btn.click()
#         while True:
#             try:        
#                 download_btn = driver.find_element(By.XPATH, '//*[@id="apiSrvTbTop"]/tbody/tr[6]/td[1]/a[1]')
#                 download_btn.click()                
#                 tolist_btn = driver.find_element(By.XPATH, '//*[@id="mainForm"]/div[2]/input')
#                 tolist_btn.click()
#             except:
#                 continue
#             break                

#     if page % 10 == 1:
#         time.sleep(1)
#         next_btn = driver.find_element(By.XPATH, f'//*[@id="next"]')
#         next_btn.click()

#     time.sleep(1)
#     page_btn = driver.find_element(By.XPATH, f'//*[@id="{page}"]')
#     page_btn.click()

# driver.quit()


In [28]:
# def csv_to_parquet(file_name):
#     read_path = os.path.join(DATA_PATH, f'train_data_csv/{file_name}.csv')
#     save_path = os.path.join(DATA_PATH, f'train_data_parquet/{file_name}.parquet')

#     df = pd.read_csv(read_path, encoding='cp949')
#     df.to_parquet(save_path)
#     del df
#     gc.collect()
#     print(file_name, 'Done.')


# path = os.path.join(DATA_PATH, 'train_data_csv')
# for (root, dirs, files) in os.walk(path):
#     for file in files:
#         file_name, _ = os.path.splitext(file)
#         csv_to_parquet(file_name)

## Load DataSet

In [50]:
target_name = 'num'
origin_age = ['PEPL', 'YNGBGS', 'CHILD']
origin_time = ['ATTEND', 'WKDAY_NOON', 'LVFFC', 'RESTDE_NOON']
origin_how = ['TKON', 'GFF']
origin_col = [
'BUSSTAT_BSNM_ID', 'BUSSTAT_BSNM_ROUTEID', 'month', 'weekday', 

'BUSSTAT_TKON_PEPL_ATTEND', 'BUSSTAT_TKON_PEPL_WKDAY_NOON',
'BUSSTAT_TKON_PEPL_LVFFC', 'BUSSTAT_TKON_PEPL_RESTDE_NOON',
'BUSSTAT_TKON_MINOR_ATTEND', 'BUSSTAT_TKON_MINOR_WKDAY_NOON',
'BUSSTAT_TKON_MINOR_LVFFC', 'BUSSTAT_TKON_MINOR_RESTDE_NOON',              

'BUSSTAT_GFF_PEPL_ATTEND', 'BUSSTAT_GFF_PEPL_WKDAY_NOON',
'BUSSTAT_GFF_PEPL_LVFFC', 'BUSSTAT_GFF_PEPL_RESTDE_NOON',
'BUSSTAT_GFF_MINOR_ATTEND', 'BUSSTAT_GFF_MINOR_WKDAY_NOON',
'BUSSTAT_GFF_MINOR_LVFFC', 'BUSSTAT_GFF_MINOR_RESTDE_NOON',                       
]
new_age = ['norm', 'minor']
new_time = ['work', 'wdnoon', 'quit', 'wenoon']
new_how = ['tkon', 'tkoff']
new_col = [
    'bstop', 'bus', 'month', 'weekday',

    'tkon_norm_work', 'tkon_norm_wdnoon', 'tkon_norm_quit', 'tkon_norm_wenoon',
    'tkon_minor_work', 'tkon_minor_wdnoon', 'tkon_minor_quit', 'tkon_minor_wenoon',

    'tkoff_norm_work', 'tkoff_norm_wdnoon', 'tkoff_norm_quit', 'tkoff_norm_wenoon',
    'tkoff_minor_work', 'tkoff_minor_wdnoon', 'tkoff_minor_quit', 'tkoff_minor_wenoon',
]

def id_statistic(train, test, ID, col1, col2):
    # train mean, sum
    train_mean = train.groupby([ID])[target_name].agg([(col1, 'mean')]).reset_index()
    train_sum = train.groupby([ID])[target_name].agg([(col2, 'sum')]).reset_index()
    train_mean_sum = pd.merge(train_mean, train_sum, on=ID)

    # merge
    train_res = pd.merge(train, train_mean_sum, how='left', on=ID)
    test_res = pd.merge(test, train_mean_sum, how='left', on=ID)
    
    return train_res, test_res         


def make_target(df, col):
    how, age, when = col.split('_')
    
    res_df = pd.DataFrame({
        'how' : [how] * len(df[col]),
        'age' : [age] * len(df[col]),
        'when' : [when] * len(df[col]),
        'num' : df[col]
    })

    return pd.concat([df.iloc[:,:4], res_df], axis=1)

def makeCol(train, test,  col_list, i):
    datas = list(itertools.combinations(col_list, i))
    for data in datas:
        col = []
        train_tmp=pd.Series(['']*len(train))
        test_tmp=pd.Series(['']*len(test))

        for j in range(i):
            col.append(data[j])
            split = '_'
            if j == 0:
                split = ''
            train_tmp = train_tmp + split + train[ data[j] ]
            test_tmp = test_tmp + split + test[ data[j] ]

        col = '_'.join(col)
        train[col] = train_tmp
        test[col] = test_tmp

        train, test = id_statistic(train, test, col, 'mean_'+col, 'sum_'+col)  

    del col
    del train_tmp
    del test_tmp

    return train, test

def make_dateset(df, how):
    # TKON = TKCAR + TRNSIT
    for a in origin_age:
        for t in origin_time:
            df[f'BUSSTAT_TKON_{a}_{t}'] = df[f'BUSSTAT_TKCAR_{a}_{t}'] + df[f'BUSSTAT_TRNSIT_{a}_{t}']

    # MINOR = YNGBGS + CHILD
    for h in origin_how:
        for t in origin_time:
            df[f'BUSSTAT_{h}_MINOR_{t}'] = df[f'BUSSTAT_{h}_YNGBGS_{t}'] + df[f'BUSSTAT_{h}_CHILD_{t}']

    df['BUSSTAT_BSNM_ID'] = df['BUSSTAT_BSNM_ID'].astype(str)
    df['BUSSTAT_BSNM_ROUTEID'] = df['BUSSTAT_BSNM_ROUTEID'].astype(str)
    date_col = pd.to_datetime(df.BUSSTAT_DT, format='%Y%m%d')
    df['month'] = date_col.apply(lambda x: x.month)
    df['weekday'] = date_col.dt.weekday

    # Select Columns
    df = df[origin_col]
    df.columns = new_col

    # make target
    data = pd.DataFrame()
    for a in new_age:
        for t in new_time:
            for h in new_how:
                data = pd.concat([data, make_target(df, f'{h}_{a}_{t}')])

    # select col / split
    col = [
        'bstop', 'bus', 'month', 'weekday',
        'how', 'age', 'when', 'num'
    ]        
    train, test = train_test_split(data[col], test_size=0.3, shuffle=True, random_state=123)            


    # feature engineering
    col_list = ['bstop', 'bus', 'month', 'weekday', 'age', 'when']
    for col in col_list:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)

    for i in range(1,7):
         train, test = makeCol(train, test, col_list, i)

    # select how
    train = train[train.how == how]
    test = test[test.how == how]

    return train, test

In [41]:
train['bus'] = train['bus'].astype(str)

In [45]:
train = pd.read_parquet( 'C:/Users/USER/Desktop/Bbeojung.kr/Data/train_data_parquet/2022년 03월 07일 교통카드 이용통계.parquet')
make_dateset(train, 'tkon')

ValueError: cannot reindex on an axis with duplicate labels

In [46]:
train = pd.read_parquet( 'C:/Users/USER/Desktop/Bbeojung.kr/Data/train_data_parquet/2022년 03월 07일 교통카드 이용통계.parquet').iloc[:,:7]
test = pd.read_parquet( 'C:/Users/USER/Desktop/Bbeojung.kr/Data/train_data_parquet/2022년 03월 07일 교통카드 이용통계.parquet').iloc[:,:7]
cols = ['bstop', 'bus', 'month', 'weekday', 'age', 'when', 'num']
col_list = ['bstop', 'bus', 'month', 'weekday', 'age', 'when']
train.columns = cols
test.columns = cols

for col in col_list:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

In [47]:
def makeCol(train, test,  col_list, i):
    datas = list(itertools.combinations(col_list, i))
    for data in datas:
        col = []
        train_tmp=pd.Series(['']*len(train))
        test_tmp=pd.Series(['']*len(test))

        for j in range(i):
            col.append(data[j])
            split = '_'
            if j == 0:
                split = ''
            train_tmp = train_tmp + split + train[ data[j] ]
            test_tmp = test_tmp + split + test[ data[j] ]

        col = '_'.join(col)
        train[col] = train_tmp
        test[col] = test_tmp

        train, test = id_statistic(train, test, col, 'mean_'+col, 'sum_'+col)  

    del col
    del train_tmp
    del test_tmp

    return train, test

In [48]:
for i in range(1,7):
    train, test = makeCol(train, test, col_list, i)

In [49]:
train.columns.tolist()

['bstop',
 'bus',
 'month',
 'weekday',
 'age',
 'when',
 'num',
 'mean_bstop',
 'sum_bstop',
 'mean_bus',
 'sum_bus',
 'mean_month',
 'sum_month',
 'mean_weekday',
 'sum_weekday',
 'mean_age',
 'sum_age',
 'mean_when',
 'sum_when',
 'bstop_bus',
 'mean_bstop_bus',
 'sum_bstop_bus',
 'bstop_month',
 'mean_bstop_month',
 'sum_bstop_month',
 'bstop_weekday',
 'mean_bstop_weekday',
 'sum_bstop_weekday',
 'bstop_age',
 'mean_bstop_age',
 'sum_bstop_age',
 'bstop_when',
 'mean_bstop_when',
 'sum_bstop_when',
 'bus_month',
 'mean_bus_month',
 'sum_bus_month',
 'bus_weekday',
 'mean_bus_weekday',
 'sum_bus_weekday',
 'bus_age',
 'mean_bus_age',
 'sum_bus_age',
 'bus_when',
 'mean_bus_when',
 'sum_bus_when',
 'month_weekday',
 'mean_month_weekday',
 'sum_month_weekday',
 'month_age',
 'mean_month_age',
 'sum_month_age',
 'month_when',
 'mean_month_when',
 'sum_month_when',
 'weekday_age',
 'mean_weekday_age',
 'sum_weekday_age',
 'weekday_when',
 'mean_weekday_when',
 'sum_weekday_when',
 'age

In [ ]:
def make_dateset(df, how):
    # feature engineering
    col_list = ['bstop', 'bus', 'month', 'weekday', 'age', 'when']
    for i in range(1,7):
         train, test = makeCol(train, test, col_list, 2)

    # select how
    train = train[train.how == how]
    test = test[test.how == how]

    return train, test

In [5]:
train = pd.DataFrame()
test = pd.DataFrame()
data_path = os.path.join(DATA_PATH, 'train_data_parquet')
sample_num = 100

for (root, dirs, files) in os.walk(data_path):
    for file in tqdm(files):
        tmp_df = pd.read_parquet( os.path.join(data_path, file) )
        tmp_df = tmp_df.iloc[random.sample(tmp_df.index.tolist(), sample_num if sample_num != 0 else len(tmp_df)),:]

        train_tmp, test_tmp = make_dateset( tmp_df, 'tkon' )
        train = pd.concat([train, train_tmp])
        test = pd.concat([test, test_tmp])

        del tmp_df, train_tmp, test_tmp
        gc.collect()

  0%|          | 0/220 [00:00<?, ?it/s]


ValueError: cannot reindex on an axis with duplicate labels

In [32]:
train

,bstop,bus,month,weekday,how,age,when,num,bstop_bus,mean_bstop_bus,...,mean_bus,sum_bus,mean_month,sum_month,mean_weekday,sum_weekday,mean_age,sum_age,mean_when,sum_when
0,1390,36,3,0,tkon,norm,work,3,1390_36,0.583333,...,0.500000,43,2.003170,44871,2.003170,44871,3.472118,38978,2.530308,14276
1,175,202,3,0,tkon,norm,work,1,175_202,0.090909,...,0.188889,17,2.003170,44871,2.003170,44871,3.472118,38978,2.530308,14276
2,1136,13,3,0,tkon,minor,wdnoon,0,1136_13,0.000000,...,3.036496,832,2.003170,44871,2.003170,44871,0.527385,5893,1.607994,8971
3,282,44,3,0,tkon,norm,quit,1,282_44,1.583333,...,1.852740,541,2.003170,44871,2.003170,44871,3.472118,38978,2.268962,12654
7,3650,95,3,0,tkon,minor,quit,0,3650_95,1.666667,...,1.702857,298,2.003170,44871,2.003170,44871,0.527385,5893,2.268962,12654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22389,1716,38,10,2,tkon,norm,wenoon,0,1716_38,0.076923,...,2.142857,465,2.309687,51737,2.309687,51737,4.047657,45439,1.842378,10321
22394,1251,92,10,2,tkon,minor,wdnoon,0,1251_92,4.428571,...,1.234043,290,2.309687,51737,2.309687,51737,0.563630,6298,1.821115,10160
22395,347,30,10,2,tkon,minor,wenoon,0,347_30,0.777778,...,1.967611,486,2.309687,51737,2.309687,51737,0.563630,6298,1.842378,10321
22396,926,10,10,2,tkon,minor,work,0,926_10,1.666667,...,2.553254,863,2.309687,51737,2.309687,51737,0.563630,6298,2.943637,16608


In [ ]:
train.to_parquet(os.path.join(DATA_PATH, f'final_dataset/train_{sample_num}.parquet'))
train.to_parquet(os.path.join(DATA_PATH, f'final_dataset/test_{sample_num}.parquet'))

In [ ]:
col = [
    'mean_bstop_bus', 'sum_bstop_bus', 'mean_bstop_month', 'sum_bstop_month',
    'mean_bstop_weekday', 'sum_bstop_weekday', 'mean_bstop_age', 'sum_bstop_age',
    'mean_bstop_when', 'sum_bstop_when', 'mean_bus_bstop', 'sum_bus_bstop',
    'mean_bus_month', 'sum_bus_month', 'mean_bus_weekday', 'sum_bus_weekday',
    'mean_bus_age', 'sum_bus_age', 'mean_bus_when', 'sum_bus_when',
    'mean_month_bstop', 'sum_month_bstop', 'mean_month_bus', 'sum_month_bus',
    'mean_month_weekday', 'sum_month_weekday', 'mean_month_age', 'sum_month_age',
    'mean_month_when', 'sum_month_when', 'mean_weekday_bstop', 'sum_weekday_bstop',
    'mean_weekday_bus', 'sum_weekday_bus', 'mean_weekday_month', 'sum_weekday_month',
    'mean_weekday_age', 'sum_weekday_age', 'mean_weekday_when', 'sum_weekday_when',
    'mean_age_bstop', 'sum_age_bstop', 'mean_age_bus', 'sum_age_bus',
    'mean_age_month', 'sum_age_month', 'mean_age_weekday', 'sum_age_weekday',
    'mean_age_when', 'sum_age_when', 'mean_when_bstop', 'sum_when_bstop',
    'mean_when_bus', 'sum_when_bus', 'mean_when_month', 'sum_when_month',
    'mean_when_weekday', 'sum_when_weekday', 'mean_when_age', 'sum_when_age',
    'mean_bstop', 'sum_bstop', 'mean_bus', 'sum_bus', 'mean_month', 'sum_month', 'mean_weekday', 'sum_weekday', 'mean_age', 'sum_age', 'mean_when', 'sum_when',
    'num'
    ]

In [ ]:
train

,mean_bstop_bus,sum_bstop_bus,mean_bstop_month,sum_bstop_month,mean_bstop_weekday,sum_bstop_weekday,mean_bstop_age,sum_bstop_age,mean_bstop_when,sum_bstop_when,...,sum_bus,mean_month,sum_month,mean_weekday,sum_weekday,mean_age,sum_age,mean_when,sum_when,num
0,0.250000,3,0.250000,3,0.250000,3,0.5,2,0.333333,1,...,27,1.933393,21654,1.933393,21654,3.323247,18670,1.520767,4284,1
1,0.363636,4,1.304348,30,1.304348,30,1.5,18,2.333333,14,...,22,1.933393,21654,1.933393,21654,3.323247,18670,2.551537,7055,2
2,1.818182,20,1.818182,20,1.818182,20,3.0,18,1.333333,4,...,348,1.933393,21654,1.933393,21654,3.323247,18670,1.520767,4284,4
3,0.090909,1,0.080000,2,0.080000,2,0.0,0,0.333333,2,...,9,1.933393,21654,1.933393,21654,0.534575,2984,2.098390,5865,0
5,1.166667,14,1.166667,14,1.166667,14,0.0,0,2.000000,8,...,38,1.933393,21654,1.933393,21654,0.534575,2984,2.551537,7055,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11184,10.111111,91,10.111111,91,10.111111,91,15.0,90,4.666667,14,...,406,2.427232,27185,2.427232,27185,4.282485,24059,1.941072,5468,9
11185,0.090909,1,0.047619,1,0.047619,1,0.1,1,0.000000,0,...,80,2.427232,27185,2.427232,27185,4.282485,24059,3.134901,8668,0
11190,3.400000,34,3.400000,34,3.400000,34,1.8,9,2.750000,11,...,324,2.427232,27185,2.427232,27185,0.560014,3126,1.941072,5468,1
11193,3.000000,33,1.750000,35,1.750000,35,0.3,3,0.333333,2,...,437,2.427232,27185,2.427232,27185,0.560014,3126,1.941072,5468,0
